# Training Data

We will create a all the possible way the user can ask the bot to play chess move. Convert that data into a text and convert text to audio for training the bot. 

This will helps us create enough dataset to train the NN.



Set the number of audio data this will create {total_training_data} audio files under training/audio and the file name will be the row of training.text file

In [185]:
total_training_data = 100_000
data_fill_all_place = True

if data_fill_all_place:
  if total_training_data < 5000:
    raise "too little training data recommend atleast 5000 for testing if data_fill_all_place is True"

In [186]:
# imports
import os
import random
from tqdm import tqdm

In [187]:
# Default variable
_random_rank2 = ["a2", "b2", "c2", "d2", "e2", "f2", "g2", "h2"]
_random_rank4 = ["a4", "b4", "c4", "d4", "e4", "f4", "g4", "h4"]
_random_rank5 = ["a5", "b5", "c5", "d5", "e5", "f5", "g5", "h5"]
_random_rank7 = ["a7", "b7", "c7", "d7", "e7", "f7", "g7", "h7"]

_random_piece = ["pawn", "knight", "bishop", "rook", "queen", "king"]
_random_power_piece = ["knight", "bishop", "rook", "queen", "king"]

_from = [ 
  "a1", "b1", "c1", "d1", "e1", "f1", "g1", "h1",
  "a2", "b2", "c2", "d2", "e2", "f2", "g2", "h2",
  "a3", "b3", "c3", "d3", "e3", "f3", "g3", "h3",
  "a4", "b4", "c4", "d4", "e4", "f4", "g4", "h4",
  "a5", "b5", "c5", "d5", "e5", "f5", "g5", "h5",
  "a6", "b6", "c6", "d6", "e6", "f6", "g6", "h6",
  "a7", "b7", "c7", "d7", "e7", "f7", "g7", "h7",
  "a8", "b8", "c8", "d8", "e8", "f8", "g8", "h8",
]

_to = [
  "a1", "b1", "c1", "d1", "e1", "f1", "g1", "h1",
  "a2", "b2", "c2", "d2", "e2", "f2", "g2", "h2",
  "a3", "b3", "c3", "d3", "e3", "f3", "g3", "h3",
  "a4", "b4", "c4", "d4", "e4", "f4", "g4", "h4",
  "a5", "b5", "c5", "d5", "e5", "f5", "g5", "h5",
  "a6", "b6", "c6", "d6", "e6", "f6", "g6", "h6",
  "a7", "b7", "c7", "d7", "e7", "f7", "g7", "h7",
  "a8", "b8", "c8", "d8", "e8", "f8", "g8", "h8",
]


In [188]:
template_training_data = [
  # by place
    "_from _to",
    "_from to _to",
    "from _from to _to",
    "move _from to _to",
    "_from capture _to",
    "capture _to",
    "_from takes _to",
    "takes _to",
  # pawn
    # pawn double move
    "_random_rank4", # Pawn move from e2 to e4
    "_random_rank5", # Pawn move from e7 to e5
    "pawn _random_rank4",
    "pawn _random_rank5",
    "_random_rank2 to _random_rank4",
    "_random_rank7 to _random_rank5",
    "pawn _random_rank2 to _random_rank4",
    "pawn _random_rank7 to _random_rank5",
    # promote only rank 7 and rank
    "pawn _random_rank7 promote _random_power_piece",
    "pawn _random_rank7 promote to _random_power_piece",
    "pawn _random_rank2 promote _random_power_piece",
    "pawn _random_rank2 promote to _random_power_piece",
  # by piece
    "move _random_piece _from _to",
    "move _random_piece _to",
    "move _random_piece _to",
    "_random_piece _from _to",
    "_random_piece _to",
    "_random_piece _to",
    "_random_piece _from to _to",
    "_random_piece to _to",
    "_random_piece to _to",
    # captures
    "capture _to _random_piece",
    "takes _to _random_piece",
    
    "_random_piece _from capture _to",
    "_random_piece capture _to",
    "_random_piece capture _to _random_piece",
    
    "_random_piece _from takes _to",
    "_random_piece takes _to",
    "_random_piece takes _to _random_piece",
    
    
    # The enemy piece comes first... helps tag which is the user piece and which is enemy
    "attack _random_piece using _random_piece",
    "attack _random_piece using _from"
]
  
# run this only 10% training data - less data is required
template_castle = [
  "long side castle",
  "king side castle",
  "castle",
  "short side castle",
  "queen side castle"
]

# en passant
template_en_passant = [
  "en passant _random_rank4",
  "en passant _random_rank5",
  "pawn en passant _random_rank4",
  "pawn en passant _random_rank5",
]


In [189]:
# Helper functions
def count_lines(file_path):
  try:
    with open(file_path, 'r') as file:
      lines = file.readlines()
      return len(lines)
  except FileNotFoundError:
    print(f"File '{file_path}' not found.")
    return 0
  
def fill_template(text):
  while "_" in text:
    text = text.replace("_from", random.choice(_from), 1)
    text = text.replace("_to", random.choice(_to), 1)
    text = text.replace("_random_rank2", random.choice(_random_rank2), 1)
    text = text.replace("_random_rank4", random.choice(_random_rank4), 1)
    text = text.replace("_random_rank5", random.choice(_random_rank5), 1)
    text = text.replace("_random_rank7", random.choice(_random_rank7), 1)
    text = text.replace("_random_piece", random.choice(_random_piece), 1)
    text = text.replace("_random_power_piece", random.choice(_random_power_piece), 1)
  return text

In [190]:
file_dir = "../training"
training_path = f"{file_dir}/training.txt"

def truncate_data(path):
  with open(path, 'w') as file:
    file.truncate()

def append_data(content):
  has_content = count_lines(training_path) != 0
  with open(training_path, 'a') as file:
    if has_content:
      content = '\n' + content
    file.write(content)
    has_content = True
  

# Create the directory if it doesn't exist
if os.path.exists(file_dir):
  truncate_data(training_path)
else:
  os.makedirs(file_dir)


In [191]:
training_data = []

In [192]:

# Fill the training data with all the places a1 to h8
if data_fill_all_place:
  len_from = len(_from)
  for i in range(len_from):
    training_data.append(_from[i])
    for j in range(len_from):
      training_data.append(f"{_from[i]} to {_to[j]}")


In [193]:
MAX_RETRY_RANDOM_TEXT = 1000
def random_training_text(CURR_RETRY=0):
  if MAX_RETRY_RANDOM_TEXT < CURR_RETRY:
    raise Exception("Max retries reached to get random text. Try lowering the text data")
  
  text = ""
  random_decimal = round(random.uniform(0, 1), 2)

  # 10% training data contains about castle
  if random_decimal < 0.2:
    text = random.choice(template_castle)
  # 10% training data contains about en passant
  elif random_decimal < 0.25:
    text = random.choice(template_en_passant)
  else:
    text = random.choice(template_training_data)
    
  # 5% chance the move is a check
  if random_decimal < 0.05:
    text = text + " with check"
  
  # Remove template
  text = fill_template(text)
  # If already there find a new one
  if text in training_data:
    return random_training_text(CURR_RETRY = CURR_RETRY + 1)
  CURR_RETRY = 0
  return text

In [198]:
len(training_data)

100000

In [195]:

left = total_training_data - len(training_data)

with tqdm(total=left) as pbar:
  while len(training_data) < total_training_data:
    training_data.append(random_training_text())
    pbar.update(1)

len_training_data = len(training_data)


  1%|          | 1008/95840 [00:00<00:09, 10076.67it/s]

100%|██████████| 95840/95840 [03:07<00:00, 511.72it/s] 


In [197]:
append_data('\n'.join(training_data))
print(f"All training text generated total text check {training_path} : {len(training_data)}.")

All training text generated total text check ../training/training.txt : 100000.
